<a href="https://colab.research.google.com/github/aditiverma18/spam_classification/blob/main/spam_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import numpy as np
import pandas as pd

df=pd.read_csv('spam.zip',encoding='latin-1')

print(df.head())

     v1                                                 v2 Unnamed: 2  \
0   ham  Go until jurong point, crazy.. Available only ...        NaN   
1   ham                      Ok lar... Joking wif u oni...        NaN   
2  spam  Free entry in 2 a wkly comp to win FA Cup fina...        NaN   
3   ham  U dun say so early hor... U c already then say...        NaN   
4   ham  Nah I don't think he goes to usf, he lives aro...        NaN   

  Unnamed: 3 Unnamed: 4  
0        NaN        NaN  
1        NaN        NaN  
2        NaN        NaN  
3        NaN        NaN  
4        NaN        NaN  


In [4]:
cols=df.columns[:2]
data=df[cols]
print(data.shape)

(5572, 2)


In [5]:
data=data.rename(columns={'v1':'Value','v2':'Text'})
print(data.head())
print(data.Value.value_counts())

  Value                                               Text
0   ham  Go until jurong point, crazy.. Available only ...
1   ham                      Ok lar... Joking wif u oni...
2  spam  Free entry in 2 a wkly comp to win FA Cup fina...
3   ham  U dun say so early hor... U c already then say...
4   ham  Nah I don't think he goes to usf, he lives aro...
Value
ham     4825
spam     747
Name: count, dtype: int64


In [6]:
from string import punctuation
import re
import nltk
nltk.download('punkt_tab')
nltk.download('words')
from nltk import wordpunct_tokenize

punctuation=list(punctuation)

data["Punctuations"]=data["Text"].apply(lambda x:len(re.findall(r"[^\w+&&^\s]",x)))

data["Phonenumbers"] = data["Text"].apply(lambda x: len(re.findall(r"[0-9]{10}",x)))

is_link = lambda x: 1 if re.search(r"https?://(?:[-\w.]|(?:%[\da-fA-F]{2}))+",x)!=None else 0
data["Links"] = data["Text"].apply(is_link)


count_upper = lambda x : list(map(str.isupper,x.split())).count(True)
upper_case = lambda y,n : n+1 if y.isupper() else n
data["Uppercase"] = data["Text"].apply(count_upper)

english_vocab_set=set(w.lower() for w in nltk.corpus.words.words())

def unusual_words(text):
    words = wordpunct_tokenize(text)
    text_vocab_set = set(w.lower() for w in words if w.isalpha())
    unusual_words_set = text_vocab_set - english_vocab_set
    return len(unusual_words_set)
data["unusualwords"]=data["Text"].apply(lambda x:unusual_words((x)))

print(data[13:25])

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.
/tmp/ipython-input-6-655865756.py:10: FutureWarning: Possible set intersection at position 5
  data["Punctuations"]=data["Text"].apply(lambda x:len(re.findall(r"[^\w+&&^\s]",x)))


   Value                                               Text  Punctuations  \
13   ham  I've been searching for the right words to tha...             4   
14   ham                I HAVE A DATE ON SUNDAY WITH WILL!!             2   
15  spam  XXXMobileMovieClub: To use your credit, click ...            11   
16   ham                         Oh k...i'm watching here:)             6   
17   ham  Eh u remember how 2 spell his name... Yes i di...             5   
18   ham  Fine if thatåÕs the way u feel. ThatåÕs the wa...             1   
19  spam  England v Macedonia - dont miss the goals/team...             7   
20   ham          Is that seriously how you spell his name?             1   
21   ham  IÛ÷m going to try for 2 months ha ha only joking             2   
22   ham  So Ì_ pay first lar... Then when is da stock c...             6   
23   ham  Aft i finish my lunch then i go str down lor. ...             3   
24   ham  Ffffffffff. Alright no way I can meet up with ...             2   

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf_idf=TfidfVectorizer(stop_words="english",strip_accents='ascii',max_features=300)
tf_idf_matrix=tf_idf.fit_transform(data["Text"])

In [8]:
data_extra_features=pd.concat([data,pd.DataFrame(tf_idf_matrix.toarray(),columns=tf_idf.get_feature_names_out())],axis=1)

In [9]:
from sklearn.model_selection import train_test_split
X=data_extra_features
features=X.columns.drop(["Value","Text"])
target=["Value"]
X_train,X_test,y_train,y_test=train_test_split(X[features],X[target])

In [10]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
dt = DecisionTreeClassifier(min_samples_split=40)
dt.fit(X_train,y_train)
pred = dt.predict(X_test)
print(accuracy_score(y_train, dt.predict(X_train)))
print(accuracy_score(y_test, pred))


0.9870782483847811
0.9734386216798278


In [11]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score



# Building a Naive Bayes Model
mnb = MultinomialNB()
mnb.fit(X_train,y_train)
pred_mnb = mnb.predict(X_test)
print(accuracy_score(y_test, pred_mnb))


# Building a Logistic Regression Model
lr = LogisticRegression()
lr.fit(X_train,y_train)
pred_lr = lr.predict(X_test)
print(accuracy_score(y_test, pred_lr))


0.9698492462311558


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.9791816223977028


/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [12]:
import pickle

with open("spam_model.pkl", "wb") as f:
    pickle.dump(lr, f)

with open("vectorizer.pkl", "wb") as f:
    pickle.dump(tf_idf, f)